### subjective/neutral of sentences analyzer
1. import the json file consists of speechs include "China"
2. use model to calculate the probability of subjective/neutral assign to sentences
3. restore the result to the independent json files
4. calculate the average, median and standard deviation of scores by president and year
5. visualize the result

In [47]:
#pip install transformers
#pip install torch torchvision torchaudio

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
import json
import os
import numpy as np
import pandas as pd

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cffl/bert-base-styleclassification-subjective-neutral")

In [8]:
def process_sentences(json_file_path, max_length=512):
    processed_sentences = []
    
    # read json file
    with open(json_file_path, 'r') as file:
        sentences_data = json.load(file)
    
    # iterate each sentence
    for entry in sentences_data:
        sentence = entry['sentence']
        
        # tokenize and truncate
        tokens = tokenizer(sentence, truncation=True, max_length=512, return_tensors="pt", padding=True)
        
        # convert to text
        truncated_sentence = tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)
        
        # save processed sentence
        processed_sentences.append(truncated_sentence)
    
    return processed_sentences

In [ ]:
classify = pipeline(
    task="text-classification",
    model="cffl/bert-base-styleclassification-subjective-neutral",
    return_all_scores=True,
)

In [5]:
# with open('./speech_json/37_nixon_speech.json', 'r') as file:
#     nixon_speech = json.load(file)

# with open('./speech_json/38_ford_speech.json', 'r') as file:
#     ford_speech = json.load(file)

# with open('./speech_json/39_carter_speech.json', 'r') as file:
#     carter_speech = json.load(file)

# with open('./speech_json/40_reagan_speech.json', 'r') as file:
#     reagan_speech = json.load(file)

# with open('./speech_json/41_herbertbush_speech.json', 'r') as file:
#     herbertbush_speech = json.load(file)

# with open('./speech_json/42_clinton_speech.json', 'r') as file:
#     clinton_speech = json.load(file)

# with open('./speech_json/43_walkerbush_speech.json', 'r') as file:
#     walkerbush_speech = json.load(file)

# with open('./speech_json/44_obama_speech.json', 'r') as file:
#     obama_speech = json.load(file)

# with open('./speech_json/45_trump_speech.json', 'r') as file:
#     trump_speech = json.load(file)

# with open('./speech_json/46_biden_speech.json', 'r') as file:
#     biden_speech = json.load(file)

# with open('./speech_json/47_vicepresident_biden_speech.json', 'r') as file:
#     vicepresident_biden_speech = json.load(file)



In [6]:
def calculate_scores(speech_file, output_file):
    results = []

    for entry in speech_file:
        sentence = entry['sentence']
        date = entry['date']
        tokens = tokenizer(sentence, truncation=True, max_length=512, return_tensors="pt", padding=True)
        truncated_sentence = tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)
        result = classify(truncated_sentence)
        subjective_score, neutral_score = result[0][0]['score'], result[0][1]['score']

        # Collect sentences with their scores and dates
        results.append({
            "date": date,
            "subjective_score": subjective_score,
            "neutral_score": neutral_score,
            "sentence": sentence
        })

    # Save results to a JSON file
    with open(output_file, 'w') as outfile:
        json.dump(results, outfile, indent=4)

    return results

In [ ]:
#nixon_results = calculate_scores(nixon_speech, 'nixon_scores.json')
# ford_results = calculate_scores(ford_speech, 'ford_scores.json')
# carter_results = calculate_scores(carter_speech, 'carter_scores.json')
# reagan_results = calculate_scores(reagan_speech, 'reagan_scores.json')
# herbertbush_results = calculate_scores(herbertbush_speech, 'herbertbush_scores.json')
# clinton_results = calculate_scores(clinton_speech, 'clinton_scores.json')
# walkerbush_results = calculate_scores(walkerbush_speech, 'walkerbush_scores.json')
# obama_results = calculate_scores(obama_speech, 'obama_scores.json')
# trump_results = calculate_scores(trump_speech, 'trump_scores.json')
# biden_results = calculate_scores(biden_speech, 'biden_scores.json')
# vicepresident_biden_results = calculate_scores(vicepresident_biden_speech, 'vicepresident_biden_scores.json')


In [11]:
# with open('./subjective_scores_json/nixon_scores.json', 'r') as file:
#     nixon_scores = json.load(file)

# with open('./subjective_scores_json/ford_scores.json', 'r') as file:
#     ford_scores = json.load(file)

# with open('./subjective_scores_json/carter_scores.json', 'r') as file:
#     carter_scores = json.load(file)

# with open('./subjective_scores_json/reagan_scores.json', 'r') as file:
#     reagan_scores = json.load(file)

# with open('./subjective_scores_json/herbertbush_scores.json', 'r') as file:
#     herbertbush_scores = json.load(file)

# with open('./subjective_scores_json/clinton_scores.json', 'r') as file:
#     clinton_scores = json.load(file)

# with open('./subjective_scores_json/walkerbush_scores.json', 'r') as file:
#     walkerbush_scores = json.load(file)

# with open('./subjective_scores_json/obama_scores.json', 'r') as file:
#     obama_scores = json.load(file)

# with open('./subjective_scores_json/trump_scores.json', 'r') as file:
#     trump_scores = json.load(file)

# with open('./subjective_scores_json/biden_scores.json', 'r') as file:
#     biden_scores = json.load(file)

# with open('./subjective_scores_json/vicepresident_biden_scores.json', 'r') as file:
#     vicepresident_biden_scores = json.load(file)


In [12]:
def statistics_calculator(scores, presidentname):

    subjective_scores = [score['subjective_score'] for score in scores]
    neutral_scores = [score['neutral_score'] for score in scores if score['neutral_score'] < 0.9]
    
    if not subjective_scores:
        return None, None, None
    
    average_subjective_score = round(np.mean(subjective_scores), 3)
    median_subjective_score = round(np.median(subjective_scores), 3)
    std_dev_subjective_score = round(np.std(subjective_scores), 3)
    average_neutral_score = round(np.mean(neutral_scores), 3)
    median_neutral_score = round(np.median(neutral_scores), 3)
    std_dev_neutral_score = round(np.std(neutral_scores), 3) 

    print(f"president_name: {presidentname}")
    print(f"average_subjective_score: {average_subjective_score:.3f}")
    print(f"median_subjective_score: {median_subjective_score:.3f}")
    print(f"std_dev_subjective_score: {std_dev_subjective_score:.3f}")
    print(f"average_neutral_score: {average_neutral_score:.3f}")
    print(f"median_neutral_score: {median_neutral_score:.3f}")
    print(f"std_dev_neutral_score: {std_dev_neutral_score:.3f}")

    results = {
        "president_name": presidentname,
        "average_subjective_score": average_subjective_score,
        "median_subjective_score": median_subjective_score,
        "std_dev_subjective_score": std_dev_subjective_score,
        "average_neutral_score": average_neutral_score,
        "median_neutral_score": median_neutral_score,
        "std_dev_neutral_score": std_dev_neutral_score
    }

    # Save results to a JSON file
    with open('scores_statistics.json', 'a') as outfile:
        json.dump(results, outfile, indent=4)
    
    return average_subjective_score, median_subjective_score, std_dev_subjective_score, average_neutral_score, median_neutral_score, std_dev_neutral_score



In [ ]:
# nixon_statistics = statistics_calculator(nixon_scores, "nixon")
# ford_statistics = statistics_calculator(ford_scores, "ford")
# carter_statistics = statistics_calculator(carter_scores, "carter")
# reagan_statistics = statistics_calculator(reagan_scores, "reagan")
# herbertbush_statistics = statistics_calculator(herbertbush_scores, "herbertbush")
# clinton_statistics = statistics_calculator(clinton_scores, "clinton")
# walkerbush_statistics = statistics_calculator(walkerbush_scores, "walkerbush")
# obama_statistics = statistics_calculator(obama_scores, "obama")
# trump_statistics = statistics_calculator(trump_scores, "trump")
# biden_statistics = statistics_calculator(biden_scores, "biden")
# vicepresident_biden_statistics = statistics_calculator(vicepresident_biden_scores, "vicepresident_biden")


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

with open('./scores_statistics.json', 'r') as file:
    scores_statistics = json.load(file)

# Extract data from scores_statistics   
presidents = [entry['president_name'] for entry in scores_statistics]
average_subjective_scores = [entry['average_subjective_score'] for entry in scores_statistics]
median_subjective_scores = [entry['median_subjective_score'] for entry in scores_statistics]
average_neutral_scores = [entry['average_neutral_score'] for entry in scores_statistics]
median_neutral_scores = [entry['median_neutral_score'] for entry in scores_statistics]

# Create subplots (2 columns: 1 for Subjective Scores, 1 for Neutral Scores)
fig = make_subplots(rows=1, cols=2, subplot_titles=("Subjective Scores", "Neutral Scores"))

# Plot average and median subjective scores in the first subplot
fig.add_trace(go.Scatter(x=presidents, y=average_subjective_scores, 
                         mode='lines+markers', name='Average Subjective Score',
                         marker=dict(symbol='circle', color='blue')), 
              row=1, col=1)

fig.add_trace(go.Scatter(x=presidents, y=median_subjective_scores, 
                         mode='lines+markers', name='Median Subjective Score',
                         marker=dict(symbol='square', color='red')), 
              row=1, col=1)

# Plot average and median neutral scores in the second subplot
fig.add_trace(go.Scatter(x=presidents, y=average_neutral_scores, 
                         mode='lines+markers', name='Average Neutral Score',
                         marker=dict(symbol='circle', color='green')), 
              row=1, col=2)

fig.add_trace(go.Scatter(x=presidents, y=median_neutral_scores, 
                         mode='lines+markers', name='Median Neutral Score',
                         marker=dict(symbol='square', color='orange')), 
              row=1, col=2)

# Update layout (rotate x-axis labels, add titles, etc.)
fig.update_layout(
    height=600, 
    width=1100, 
    title_text="Subjective and Neutral Sentiment Scores by President",
    title_font_size=24,
    showlegend=True,
    hovermode='closest',  # Ensures closest point to cursor triggers hover
    xaxis=dict(tickangle=-45),
    yaxis=dict(showline=True),
    xaxis2=dict(tickangle=-45),
    yaxis2=dict(showline=True),
)

fig.update_traces(hoverinfo='text', hovertemplate='%{x}<br>Score: %{y}', 
                  hoverlabel=dict(namelength=-1), 
                  marker=dict(symbol='circle', size=10),
                  line=dict(width=2), 
                  mode='lines+markers')

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=-45)

# Display the figure
fig.show()
    

In [56]:
def top10_sentences(scores):
    subjective_sentences = []
    neutral_sentences = []

    for entry in scores:
        sentence = entry['sentence']
        subjective_score = entry['subjective_score']
        neutral_score = entry['neutral_score']
        if neutral_score < 0.9:
            subjective_sentences.append((sentence, subjective_score))
            neutral_sentences.append((sentence, neutral_score))

    # Sort and print top 10 subjective and neutral sentences
    subjective_sentences.sort(key=lambda x: x[1], reverse=True)
    neutral_sentences.sort(key=lambda x: x[1], reverse=True)

    print("\nTop 10 Subjective Sentences:")
    for sentence, score in subjective_sentences[:10]:
        print(f"{score:.4f}: {sentence}")

    print("\nTop 10 Neutral Sentences:")
    for sentence, score in neutral_sentences[:10]:
        print(f"{score:.4f}: {sentence}")

In [ ]:
# top10_sentences(nixon_scores)
# top10_sentences(ford_scores)
# top10_sentences(carter_scores)
# top10_sentences(reagan_scores)
# top10_sentences(herbertbush_scores)
# top10_sentences(clinton_scores)
# top10_sentences(walkerbush_scores)
# top10_sentences(obama_scores)
# top10_sentences(trump_scores)
# top10_sentences(biden_scores)
# top10_sentences(vicepresident_biden_scores)